# <span style="color:green; font-family:Georgia; font-size:2.1m;"> 1- Modèle de prediction 1

# <span style="color:green; font-family:Georgia; font-size:1.5m;"> 1- Préparation des données

In [6]:
import numpy as np
from unidecode import unidecode
import string
from sklearn import svm
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words('french')
import spacy
from spacy_lefff import LefffLemmatizer, POSTagger
nlp = spacy.load('fr_core_news_sm')
french_lemmatizer = LefffLemmatizer()
nlp.add_pipe(french_lemmatizer, name='lefff')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mamdiallo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2019-10-17 18:40:51,904 - spacy_lefff.lefff - INFO - New LefffLemmatizer instantiated.
2019-10-17 18:40:51,906 - spacy_lefff.lefff - INFO - Reading lefff data...
2019-10-17 18:40:52,366 - spacy_lefff.lefff - INFO - Successfully loaded lefff lemmatizer


In [7]:
fichier = open("CorpusM2-AFD/corpus.tache1.learn","r",encoding = "latin-1")
lignes = fichier.readlines()

In [8]:
data = []
for ligne in lignes:
    ligne = ligne.replace('\n','')
    data.append(ligne)

In [9]:
from nltk.stem.snowball import FrenchStemmer
stemmer = FrenchStemmer()
print(data[0])
print(data[0].split(' ')[3])
print(stemmer.stem(data[0].split(' ')[3]))
print(stemmer.stem('diplomatique'))

<100:1:C> Quand je dis chers amis, il ne s'agit pas là d'une formule diplomatique, mais de l'expression de ce que je ressens.
dis
dis
diplomat


In [10]:

print(stopwords)

['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'auras', 'aura', 'aurons', 'aur

In [11]:
def zeroOuUn(c):
    if c == 'C':
        return 1
    else:
        return 0

In [12]:
def recupPersonne(s):
    for i in range(0,len(s)):
        if s[i]=='>':
            return(zeroOuUn(s[i-1]))

In [13]:
def recupDonnees():
    X = []
    y = []
    for doc in data:
        doc1 = doc.split(' ')
#         doc1 =  [stemmer.stem(m) for m in doc1] #lemmetisation
    #     print(" ".join(doc1[1:]))
        res = " ".join(doc1[1:])        
        res = res.lower() #convertit les Majuscules en miniscules
#         print(res)
        doc_lem = nlp(res)
        phrase = ""
        for d in doc_lem:
            phrase += d.lemma_ +" "
        res = phrase
        res = unidecode(res) #suppression des accents et cedilles
        res = res.translate(str.maketrans('', '', string.punctuation)) #suprression des ponctuations
        res = res.split(' ')
        res = " ".join([mot for mot in res if mot not in stopwords]) #suppression des stopwords
#         print(res)
        X.append(res)
        y.append(recupPersonne(doc1[0]))
#         break
    return X,y

In [14]:
X, y = recupDonnees()

KeyboardInterrupt: 

In [ ]:
print(X[:100])

In [ ]:
import random
SEED = 987654321
random.seed(SEED)
random.shuffle(X)
random.shuffle(y)
# print(X[0:10])
# print(y[0:10])

In [ ]:
print(y[0], " - ",X[0])
print(y[1], " - ",X[1])
print(y[12], " - ",X[12])

In [16]:
mot_max = 1500
from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer(ngram_range=(4,4))
vectorizer = TfidfVectorizer(max_features=1500,ngram_range=(3,3),min_df=3, max_df=0.7)
# vectorizer = TfidfVectorizer(max_features=mot_max, min_df=5, max_df=0.7)

X_tfifd = vectorizer.fit_transform(X).toarray()

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tfifd, y, test_size=0.2, random_state=0)
mot_max = 2500
print(len(X_train))
print(len(y_train))

print(len(X_test))
print(len(y_test))

NameError: name 'X' is not defined

In [17]:
print(vectorizer.get_feature_names())

['000', '10', '100', '11', '14', '15', '16', '1789', '20', '200', '22', '24', '25', '27', '30', '300', '320', '35', '40', '50', '60', '600', '700', '80', 'a380', 'abandon', 'abandonner', 'abord', 'aborde', 'aborder', 'aboutir', 'abri', 'absence', 'absolu', 'absolument', 'academie', 'acadien', 'acceder', 'acceleration', 'accelerer', 'accent', 'accentuer', 'acceptable', 'acceptation', 'accepte', 'accepter', 'acces', 'accessible', 'accident', 'accompagne', 'accompagnement', 'accompagner', 'accomplir', 'accomplissement', 'accord', 'accorde', 'accorder', 'accroissement', 'accroitre', 'accru', 'accueil', 'accueillir', 'accumuler', 'achat', 'achever', 'acquerir', 'acquis', 'acte', 'acteur', 'actif', 'action', 'actionnaire', 'activement', 'activite', 'actualite', 'actuel', 'actuellement', 'adaptation', 'adapte', 'adapter', 'adherer', 'adhesion', 'adjoint', 'admettre', 'administratif', 'administration', 'admirable', 'admiration', 'admire', 'admirer', 'adolescent', 'adopter', 'adoption', 'adosse

In [1]:
print(X_train.shape)
print(X_test.shape)


NameError: name 'X_train_tfifd' is not defined

In [22]:
import torch
import torch.utils.data as utils
from torchvision import datasets, transforms
import torch.nn.functional as F

In [23]:
torch.set_default_tensor_type(torch.FloatTensor)
# print(torch.get_default_dtype())
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# train_data_tensor = torch.tensor(X_train_tfifd.toarray())
train_data_tensor = torch.tensor(X_train)
train_target_tensor = torch.tensor(np.asarray(y_train)) # transform to torch tensors

# test_data_tensor = torch.tensor(X_test_tfifd.toarray())
test_data_tensor = torch.tensor(X_test)
test_target_tensor = torch.tensor(np.asarray(y_test)) # transform to torch tensors

train_dataset = utils.TensorDataset(train_data_tensor,train_target_tensor) # create your datset
test_dataset = utils.TensorDataset(test_data_tensor,test_target_tensor) # create your datset

batch_size = 16
# Set the training loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# Set the testing loader
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [24]:
weights1 = torch.randn(mot_max, 5, requires_grad=True)
weights2 = torch.randn(5, 5, requires_grad=True)
weights3 = torch.randn(5, 2, requires_grad=True)

b1 = torch.randn((1, 5), requires_grad=True) # bias for hidden layer
b2 = torch.randn((1, 5), requires_grad=True) # bias for output layer
b3 = torch.randn((1, 2), requires_grad=True) # bias for output layer

# b1 = torch.randn(500,)
# b2 = torch.randn((1, 2))

In [25]:
def sigmoid_activation(z):
    return 1 / (1 + torch.exp(-z))

In [26]:
def test(weights1, weights2,weights3,b1, b2, b3, test_loader):
    test_size = len(test_loader.dataset)
    correct = 0

    for batch_idx, (data, target) in enumerate(test_loader):
        #print(batch_idx, data.shape, target.shape)
#         data = data.view((-1, 28*28))
        #print(batch_idx, data.shape, target.shape)
        data = data.float()
        target = target.to(dtype=torch.long)        
        z1 = torch.matmul(data, weights1) + b1
        a1 = sigmoid_activation(z1)
        
        z2 = torch.matmul(a1, weights2) + b2
        a2 = sigmoid_activation(z2)

        z3 = torch.matmul(a2, weights3) + b3
        
        outputs = sigmoid_activation(z3)
        
        softmax = F.softmax(outputs, dim=1)
        pred = softmax.argmax(dim=1, keepdim=True)
        n_correct = pred.eq(target.view_as(pred)).sum().item()
        correct += n_correct

    acc = correct / test_size
    print(" Accuracy on test set", acc)
    return

test(weights1, weights2,weights3,b1, b2, b3, test_loader)

 Accuracy on test set 0.6727336062004703


In [27]:
it = 0
for batch_idx, (data, targets) in enumerate(train_loader):
    # Be sure to start the loop with zeros grad
    if weights1.grad is not None:
        weights1.grad.zero_()
    
    if weights2.grad is not None:
        weights2.grad.zero_()
        
    if weights3.grad is not None:
        weights3.grad.zero_()
    
    if b1.grad is not None:
        b1.grad.zero_()
        
    if b2.grad is not None:
        b2.grad.zero_()
        
    if b3.grad is not None:
        b3.grad.zero_()
        
        
    #print("batch_idx: {}, data.shape: {}, target.shape: {}".format(batch_idx, data.shape, targets.shape))
    data = data.float()
    targets = targets.to(dtype=torch.long)
    z1 = torch.matmul(data, weights1) + b1
    a1 = sigmoid_activation(z1)
        
    z2 = torch.matmul(a1, weights2) + b2
    a2 = sigmoid_activation(z2)

    z3 = torch.matmul(a2, weights3) + b3
        
    outputs = sigmoid_activation(z3)
    log_softmax = F.log_softmax(outputs, dim=1)
    #print("Log softmax: {}".format(log_softmax.shape))

    #print((-log_softmax[0][targets[0]] + -log_softmax[1][targets[1]] )  / 2 )
    #print(-log_softmax[0][targets[0]], targets[0])
    
    loss = F.nll_loss(log_softmax, targets)
    print("\rLoss shape: {}".format(loss), end="")
    
    # Compute the gradients for each variables
    loss.backward()
    
    with torch.no_grad():
        weights3 -= 0.1*weights3.grad
        weights2 -= 0.1*weights2.grad
        weights1 -= 0.1*weights1.grad
        b3 -= 0.1*b3.grad
        b2 -= 0.1*b2.grad
        b1 -= 0.1*b1.grad


    it += 1
    if it % 10 == 0:
        test(weights1, weights2,weights3,b1, b2, b3, test_loader)
        
#     if it > 50000:
#         break

Loss shape: 0.6601667404174805 Accuracy on test set 0.8679787511974223
Loss shape: 0.6287295222282412 Accuracy on test set 0.8679787511974223
Loss shape: 0.6079935431480408 Accuracy on test set 0.8679787511974223
Loss shape: 0.5940302610397339 Accuracy on test set 0.8679787511974223
Loss shape: 0.6015344858169556 Accuracy on test set 0.8679787511974223
Loss shape: 0.5424875617027283 Accuracy on test set 0.8679787511974223
Loss shape: 0.6087452769279485 Accuracy on test set 0.8679787511974223
Loss shape: 0.5370907783508301 Accuracy on test set 0.8679787511974223
Loss shape: 0.52741175889968877 Accuracy on test set 0.8679787511974223
Loss shape: 0.55050367116928166 Accuracy on test set 0.8679787511974223
Loss shape: 0.54726982116699227 Accuracy on test set 0.8679787511974223
Loss shape: 0.62719506025314335 Accuracy on test set 0.8679787511974223
Loss shape: 0.45647254586219794 Accuracy on test set 0.8679787511974223
Loss shape: 0.45022335648536685 Accuracy on test set 0.8679787511974223


Loss shape: 0.37924355268478394 Accuracy on test set 0.8679787511974223
Loss shape: 0.50254130363464366 Accuracy on test set 0.8679787511974223
Loss shape: 0.44079557061195374 Accuracy on test set 0.8679787511974223
Loss shape: 0.44063013792037964 Accuracy on test set 0.8679787511974223
Loss shape: 0.44073352217674255 Accuracy on test set 0.8679787511974223
Loss shape: 0.44066977500915533 Accuracy on test set 0.8679787511974223
Loss shape: 0.50232410430908246 Accuracy on test set 0.8679787511974223
Loss shape: 0.44047290086746216 Accuracy on test set 0.8679787511974223
Loss shape: 0.44040736556053164 Accuracy on test set 0.8679787511974223
Loss shape: 0.44076195359230047 Accuracy on test set 0.8679787511974223
Loss shape: 0.50233191251754766 Accuracy on test set 0.8679787511974223
Loss shape: 0.56357479095458985 Accuracy on test set 0.8679787511974223
Loss shape: 0.50204330682754526 Accuracy on test set 0.8679787511974223
Loss shape: 0.37921270728111267 Accuracy on test set 0.867978751

In [33]:
batch_idx, (data, target) = next(enumerate(test_loader))
data = data.float()
# for i in data[0]:
#     print(i)
print(target)
z1 = torch.matmul(data, weights1) + b1
a1 = sigmoid_activation(z1)       
z2 = torch.matmul(a1, weights2) + b2
a2 = sigmoid_activation(z2)
z3 = torch.matmul(a2, weights3) + b3      
outputs = sigmoid_activation(z3)
softmax = F.softmax(outputs, dim=1)
pred = softmax.argmax(dim=1, keepdim=True)
print(pred)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1], dtype=torch.int32)
tensor([[1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1]])
